In [2223]:
import numpy as np
import pandas as pd
import Quandl
from datetime import datetime
from pandas import DataFrame
%matplotlib inline
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [2224]:
#call data from Quandl to create continuous time series of spot cattle prices
Z07 = Quandl.get("CME/LCZ2007", trim_start="2007-04-01", trim_end="2007-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
M08 = Quandl.get("CME/LCM2008", trim_start="2007-10-01", trim_end="2008-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
Z08 = Quandl.get("CME/LCZ2008", trim_start="2008-04-01", trim_end="2008-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
M09 = Quandl.get("CME/LCM2009", trim_start="2008-10-01", trim_end="2009-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
Z09 = Quandl.get("CME/LCZ2009", trim_start="2009-04-01", trim_end="2009-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
M10 = Quandl.get("CME/LCM2010", trim_start="2009-10-01", trim_end="2010-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
Z10 = Quandl.get("CME/LCZ2010", trim_start="2010-04-01", trim_end="2010-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
M11 = Quandl.get("CME/LCM2011", trim_start="2010-10-01", trim_end="2011-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
Z11 = Quandl.get("CME/LCZ2011", trim_start="2011-04-01", trim_end="2011-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
M12 = Quandl.get("CME/LCM2012", trim_start="2011-10-01", trim_end="2012-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
Z12 = Quandl.get("CME/LCZ2012", trim_start="2012-04-01", trim_end="2012-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
M13 = Quandl.get("CME/LCM2013", trim_start="2012-10-01", trim_end="2013-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
Z13 = Quandl.get("CME/LCZ2013", trim_start="2013-04-01", trim_end="2013-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
M14 = Quandl.get("CME/LCM2014", trim_start="2013-10-01", trim_end="2014-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
Z14 = Quandl.get("CME/LCZ2014", trim_start="2014-04-01", trim_end="2014-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
M15 = Quandl.get("CME/LCM2015", trim_start="2014-10-01", trim_end="2015-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
Z15 = Quandl.get("CME/LCZ2015", trim_start="2015-04-01", trim_end="2015-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
M16 = Quandl.get("CME/LCM2016", trim_start="2015-10-01", trim_end="2016-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
Z16 = Quandl.get("CME/LCZ2016", trim_start="2016-04-01", trim_end="2016-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
M17 = Quandl.get("CME/LCM2017", trim_start="2016-10-01", trim_end="2017-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")

In [2225]:
frames = [Z07, M08, Z08, M09, Z09, M10, Z10, M11, Z11, M12, Z12, M13, Z13, M14, Z14, M15, Z15, M16,
       Z16, M17]
cattle = pd.concat(frames)

In [2226]:
cattle.shape

(2506, 10)

In [2240]:
cattle.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2506 entries, 2007-04-02 to 2017-03-31
Data columns (total 6 columns):
Prev. Day Open Interest       1624 non-null float64
Previous Day Open Interest    125 non-null float64
Settle                        2506 non-null float64
Volume                        2505 non-null float64
A                             2505 non-null float64
y                             2506 non-null int64
dtypes: float64(5), int64(1)
memory usage: 137.0 KB


In [2227]:
cattle.head()

,Change,High,Last,Low,Open,Open Interest,Prev. Day Open Interest,Previous Day Open Interest,Settle,Volume
Date,,,,,,,,,,
2007-04-02,NaN,97.1,NaN,96.250,96.57,NaN,14036.0,NaN,97.000,2188.0
2007-04-03,NaN,97.2,NaN,96.400,96.60,NaN,13355.0,NaN,97.150,1888.0
2007-04-04,NaN,97.6,NaN,96.650,96.90,NaN,13619.0,NaN,97.275,759.0
2007-04-05,NaN,98.0,NaN,97.500,97.50,NaN,13870.0,NaN,97.725,773.0
2007-04-09,NaN,97.8,NaN,97.175,97.60,NaN,14040.0,NaN,97.700,1018.0


In [2228]:
#drop features from dataframe that won't be included in analysis
cattle = cattle.drop(['Open', 'High', 'Low', 'Last', 'Change', 'Open Interest'], axis=1)

In [2229]:
cattle.head()

,Prev. Day Open Interest,Previous Day Open Interest,Settle,Volume
Date,,,,
2007-04-02,14036.0,NaN,97.000,2188.0
2007-04-03,13355.0,NaN,97.150,1888.0
2007-04-04,13619.0,NaN,97.275,759.0
2007-04-05,13870.0,NaN,97.725,773.0
2007-04-09,14040.0,NaN,97.700,1018.0


In [2230]:
cattle['A'] = cattle['Settle'][::]

In [2231]:
cattle.head()

,Prev. Day Open Interest,Previous Day Open Interest,Settle,Volume,A
Date,,,,,
2007-04-02,14036.0,NaN,97.000,2188.0,97.000
2007-04-03,13355.0,NaN,97.150,1888.0,97.150
2007-04-04,13619.0,NaN,97.275,759.0,97.275
2007-04-05,13870.0,NaN,97.725,773.0,97.725
2007-04-09,14040.0,NaN,97.700,1018.0,97.700


In [2232]:
#shift column one day so today's prediction is relying on yesterday's metrics
cattle['A'] = cattle['A'].shift(1)

In [2233]:
#shift column one day so today's prediction is relying on yesterday's metrics
cattle['Volume'] = cattle['Volume'].shift(1)

In [2234]:
cattle.head()

,Prev. Day Open Interest,Previous Day Open Interest,Settle,Volume,A
Date,,,,,
2007-04-02,14036.0,NaN,97.000,NaN,NaN
2007-04-03,13355.0,NaN,97.150,2188.0,97.000
2007-04-04,13619.0,NaN,97.275,1888.0,97.150
2007-04-05,13870.0,NaN,97.725,759.0,97.275
2007-04-09,14040.0,NaN,97.700,773.0,97.725


In [2235]:
list_a = cattle['Settle']
list_b = cattle['A']

In [2236]:
#create a for loop that iterates through 'Settle' and 'Previous Day Setttle' and determines if current
#day settle is higher than previous day settle.  Higher settle labeled as '1', lower settle '0'. 
l = []
for a, b in zip(list_a, list_b):
    if a > b:
        l.append(1)
    else:
        l.append(0)

In [2237]:
cattle['y'] = l

In [2238]:
cattle.head()

,Prev. Day Open Interest,Previous Day Open Interest,Settle,Volume,A,y
Date,,,,,,
2007-04-02,14036.0,NaN,97.000,NaN,NaN,0
2007-04-03,13355.0,NaN,97.150,2188.0,97.000,1
2007-04-04,13619.0,NaN,97.275,1888.0,97.150,1
2007-04-05,13870.0,NaN,97.725,759.0,97.275,1
2007-04-09,14040.0,NaN,97.700,773.0,97.725,0


In [2239]:
#since our binary target class has realtively similar counts there is no need to balance it 
pd.value_counts(cattle['y'].values)

1    1268
0    1238
dtype: int64

In [2181]:
#some columns hold the same information but are labeled differently.  Combine these 
cattle['Prev. Day Open Interest'].fillna(cattle['Previous Day Open Interest'], inplace=True)
del cattle['Previous Day Open Interest']

In [2182]:
#significant null values in Open Interest feature
cattle.isnull().sum()

Prev. Day Open Interest    757
Settle                       0
Volume                       1
A                            1
y                            0
dtype: int64

In [2183]:
#call data from Quandl to create continuous time series of spot hog prices
HZ07 = Quandl.get("CME/LNZ2007", trim_start="2007-04-01", trim_end="2007-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HM08 = Quandl.get("CME/LNM2008", trim_start="2007-10-01", trim_end="2008-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HZ08 = Quandl.get("CME/LNZ2008", trim_start="2008-04-01", trim_end="2008-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HM09 = Quandl.get("CME/LNM2009", trim_start="2008-10-01", trim_end="2009-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HZ09 = Quandl.get("CME/LNZ2009", trim_start="2009-04-01", trim_end="2009-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HM10 = Quandl.get("CME/LNM2010", trim_start="2009-10-01", trim_end="2010-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HZ10 = Quandl.get("CME/LNZ2010", trim_start="2010-04-01", trim_end="2010-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HM11 = Quandl.get("CME/LNM2011", trim_start="2010-10-01", trim_end="2011-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HZ11 = Quandl.get("CME/LNZ2011", trim_start="2011-04-01", trim_end="2011-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HM12 = Quandl.get("CME/LNM2012", trim_start="2011-10-01", trim_end="2012-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HZ12 = Quandl.get("CME/LNZ2012", trim_start="2012-04-01", trim_end="2012-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HM13 = Quandl.get("CME/LNM2013", trim_start="2012-10-01", trim_end="2013-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HZ13 = Quandl.get("CME/LNZ2013", trim_start="2013-04-01", trim_end="2013-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HM14 = Quandl.get("CME/LNM2014", trim_start="2013-10-01", trim_end="2014-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HZ14 = Quandl.get("CME/LNZ2014", trim_start="2014-04-01", trim_end="2014-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HM15 = Quandl.get("CME/LNM2015", trim_start="2014-10-01", trim_end="2015-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HZ15 = Quandl.get("CME/LNZ2015", trim_start="2015-04-01", trim_end="2015-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HM16 = Quandl.get("CME/LNM2016", trim_start="2015-10-01", trim_end="2016-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HZ16 = Quandl.get("CME/LNZ2016", trim_start="2016-04-01", trim_end="2016-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
HM17 = Quandl.get("CME/LNM2017", trim_start="2016-10-01", trim_end="2017-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")

In [2184]:
frames = [HZ07, HM08, HZ08, HM09, HZ09, HM10, HZ10, HM11, HZ11, HM12, HZ12, HM13, HZ13, HM14, HZ14,
          HM15, HZ15, HM16, HZ16, HM17]
hogs = pd.concat(frames)

In [2185]:
hogs.head()

,Change,High,Last,Low,Open,Open Interest,Prev. Day Open Interest,Previous Day Open Interest,Settle,Volume
Date,,,,,,,,,,
2007-04-02,NaN,65.55,NaN,64.500,64.500,NaN,11777.0,NaN,65.000,409.0
2007-04-03,NaN,66.25,NaN,65.225,65.350,NaN,11763.0,NaN,65.650,650.0
2007-04-04,NaN,66.25,NaN,65.500,65.500,NaN,11720.0,NaN,66.000,522.0
2007-04-05,NaN,66.55,NaN,66.000,66.000,NaN,11536.0,NaN,66.400,389.0
2007-04-09,NaN,67.40,NaN,66.850,66.925,NaN,11471.0,NaN,67.025,336.0


In [2186]:
#take 'Settle' feature from Hog dataframe and add it to Cattle dataframe
cattle['Hogs'] = hogs['Settle']

In [2187]:
#shift column one day so today's prediction is relying on yesterday's metrics
cattle['Hogs'] = cattle['Hogs'].shift(1)

In [2188]:
cattle.head()

,Prev. Day Open Interest,Settle,Volume,A,y,Hogs
Date,,,,,,
2007-04-02,14036.0,97.000,NaN,NaN,0,NaN
2007-04-03,13355.0,97.150,2188.0,97.000,1,65.00
2007-04-04,13619.0,97.275,1888.0,97.150,1,65.65
2007-04-05,13870.0,97.725,759.0,97.275,1,66.00
2007-04-09,14040.0,97.700,773.0,97.725,0,66.40


In [2189]:
#call data from Quandl to create continuous time series of spot corn prices
CZ07 = Quandl.get("CME/CZ2007", trim_start="2007-04-01", trim_end="2007-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CM08 = Quandl.get("CME/CK2008", trim_start="2007-10-01", trim_end="2008-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CZ08 = Quandl.get("CME/CZ2008", trim_start="2008-04-01", trim_end="2008-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CM09 = Quandl.get("CME/CK2009", trim_start="2008-10-01", trim_end="2009-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CZ09 = Quandl.get("CME/CZ2009", trim_start="2009-04-01", trim_end="2009-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CM10 = Quandl.get("CME/CK2010", trim_start="2009-10-01", trim_end="2010-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CZ10 = Quandl.get("CME/CZ2010", trim_start="2010-04-01", trim_end="2010-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CM11 = Quandl.get("CME/CK2011", trim_start="2010-10-01", trim_end="2011-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CZ11 = Quandl.get("CME/CZ2011", trim_start="2011-04-01", trim_end="2011-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CM12 = Quandl.get("CME/CK2012", trim_start="2011-10-01", trim_end="2012-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CZ12 = Quandl.get("CME/CZ2012", trim_start="2012-04-01", trim_end="2012-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CM13 = Quandl.get("CME/CK2013", trim_start="2012-10-01", trim_end="2013-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CZ13 = Quandl.get("CME/CZ2013", trim_start="2013-04-01", trim_end="2013-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CM14 = Quandl.get("CME/CK2014", trim_start="2013-10-01", trim_end="2014-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CZ14 = Quandl.get("CME/CZ2014", trim_start="2014-04-01", trim_end="2014-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CM15 = Quandl.get("CME/CK2015", trim_start="2014-10-01", trim_end="2015-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CZ15 = Quandl.get("CME/CZ2015", trim_start="2015-04-01", trim_end="2015-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CM16 = Quandl.get("CME/CK2016", trim_start="2015-10-01", trim_end="2016-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CZ16 = Quandl.get("CME/CZ2016", trim_start="2016-04-01", trim_end="2016-09-30",
                   authtoken="V59QDSuBq4zt1ofAQxRv")
CM17 = Quandl.get("CME/CK2017", trim_start="2016-10-01", trim_end="2017-03-31",
                   authtoken="V59QDSuBq4zt1ofAQxRv")

In [2190]:
frames = [CZ07, CM08, CZ08, CM09, CZ09, CM10, CZ10, CM11, CZ11, CM12, CZ12, CM13, CZ13, CM14, CZ14, 
          CM15, CZ15, CM16, CZ16, CM17]
corn = pd.concat(frames)

In [2191]:
#take 'Settle' feature from Corn dataframe and add it to Cattle dataframe
cattle['Corn'] = corn['Settle']

In [2192]:
#shift column one day so today's prediction is relying on yesterday's metrics
cattle['Corn'] = cattle['Corn'].shift(1)

In [2193]:
cattle.head()

,Prev. Day Open Interest,Settle,Volume,A,y,Hogs,Corn
Date,,,,,,,
2007-04-02,14036.0,97.000,NaN,NaN,0,NaN,NaN
2007-04-03,13355.0,97.150,2188.0,97.000,1,65.00,369.50
2007-04-04,13619.0,97.275,1888.0,97.150,1,65.65,367.50
2007-04-05,13870.0,97.725,759.0,97.275,1,66.00,380.50
2007-04-09,14040.0,97.700,773.0,97.725,0,66.40,386.75


In [2194]:
cattle.isnull().sum()

Prev. Day Open Interest    757
Settle                       0
Volume                       1
A                            1
y                            0
Hogs                         2
Corn                         1
dtype: int64

In [2195]:
#we will attempt to predict cattle prices for the last full year of the data.  We will use the previous
#9 years as the training dataset
cattle_train = cattle[:'2016-03-31']
cattle_test = cattle['2016-04-01':]

In [2196]:
cattle_train.shape

(2253, 7)

In [2197]:
cattle_test.shape

(253, 7)

In [2198]:
#drop any rows of training data with null values
cattle_train = cattle_train.dropna()

In [2199]:
#drop any rows of test data with null values
cattle_test = cattle_test.dropna()

In [2200]:
cattle_train.shape

(1622, 7)

In [2201]:
cattle_test.shape

(125, 7)

In [2202]:
cattle_train = cattle_train.rename(index=str, columns = {'A' : 'Prev. Day Settle' })

In [2203]:
cattle_test = cattle_test.rename(index=str, columns = {'A' : 'Prev. Day Settle' })

In [2204]:
#reorder features in dataframe
cattle_train_X = cattle_train[['Settle', 'Prev. Day Settle', 'Volume', 'Prev. Day Open Interest',
                            'Hogs', 'Corn']]

In [2205]:
cattle_train_y = cattle_train['y']

In [2206]:
#reorder features in dataframe
cattle_test_X = cattle_test[['Settle', 'Prev. Day Settle', 'Volume', 'Prev. Day Open Interest',
                            'Hogs', 'Corn']]

In [2207]:
cattle_test_y = cattle_test['y']

In [2208]:
#apply standard scaler to X features to account for difference in Euclidian distance between 
#non-standardized X features.  y is categorical so no need to scale. 
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
cattle_train_X = sc_X.fit_transform(cattle_train_X)
cattle_test_X = sc_X.transform(cattle_test_X)

In [2209]:
#fitting naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
NBclassifier = GaussianNB()
NBclassifier.fit(cattle_train_X, cattle_train_y)

GaussianNB(priors=None)

In [2210]:
NBclassifier.score(cattle_train_X, cattle_train_y)

0.5271270036991369

In [2211]:
NBclassifier.score(cattle_test_X, cattle_test_y)

0.568

In [2212]:
#define fucntion to run a baseline score for classifiers using default hyperparameter values
def scores(models, X, y):
    for model in models:
        y_pred = model.predict(X)
        acc = accuracy_score(y, y_pred)
        print("Accuracy Score: {0:0.2f} %".format(acc * 100))

In [2213]:
#classification models used with default hyperparameters
models = [LogisticRegression(random_state=13),
          KNeighborsClassifier(),
          SVC(random_state = 13),
          RandomForestClassifier(random_state = 13)]

In [2214]:
for model in models:
    model.fit(cattle_train_X, cattle_train_y)

In [2215]:
scores(models, cattle_train_X, cattle_train_y)

Accuracy Score: 92.91 %
Accuracy Score: 70.16 %
Accuracy Score: 58.45 %
Accuracy Score: 97.78 %


In [2216]:
#logistic regression shows promise with good generalization using default parameters
#KNN and RandomForest training models seem to overfit and not generalize well
scores(models, cattle_test_X, cattle_test_y)

Accuracy Score: 91.20 %
Accuracy Score: 51.20 %
Accuracy Score: 58.40 %
Accuracy Score: 55.20 %


In [2217]:
#define hyperparameters to be used during GridSearch
LR_grid_parameters = {'C': [0.1, 1, 10, 50], 'penalty': ['l1', 'l2'], 'random_state': [13]}
KNN_grid_parameters = {'n_neighbors': [3, 5, 10, 20], 'weights': ['uniform', 'distance']}
SVM_poly_grid_parameters = {'kernel': ['poly'], 'degree': [1, 2, 3, 4], 'gamma': [.5, .2, .05, .01, .001],
                            'C': [0.1, 1, 10, 50], 'random_state': [13]}
SVM_grid_parameters = {'kernel': ['linear', 'rbf', 'sigmoid'], 'gamma': [.5, .2, .05, .01, .001],
                       'C': [0.1, 1, 10, 50], 'random_state': [13]}
RF_grid_parameters = {'n_estimators': [10, 50, 100], 'criterion': ['gini', 'entropy'],
               'max_depth': [None, 10, 25, 50], 'random_state': [13]}

In [2218]:
LR_grid = GridSearchCV(estimator = LogisticRegression(), param_grid = LR_grid_parameters, 
                        scoring = 'accuracy')
LR_grid.fit(cattle_train_X, cattle_train_y)
print("Best score: %0.3f" % LR_grid.best_score_)
print("Best parameters set:")
best_parameters=LR_grid.best_estimator_.get_params()
for param_name in sorted(LR_grid_parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.990
Best parameters set:
	C: 50
	penalty: 'l1'
	random_state: 13


In [2219]:
KNN_grid = GridSearchCV(estimator = KNeighborsClassifier(), param_grid = KNN_grid_parameters, 
                        scoring = 'accuracy')
KNN_grid.fit(cattle_train_X, cattle_train_y)
print("Best score: %0.3f" % KNN_grid.best_score_)
print("Best parameters set:")
best_parameters=KNN_grid.best_estimator_.get_params()
for param_name in sorted(KNN_grid_parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.493
Best parameters set:
	n_neighbors: 3
	weights: 'distance'


In [2220]:
SVM_poly_grid = GridSearchCV(estimator = SVC(), param_grid = SVM_poly_grid_parameters, 
                             scoring = 'accuracy')
SVM_poly_grid.fit(cattle_train_X, cattle_train_y)
print("Best score: %0.3f" % SVM_poly_grid.best_score_)
print("Best parameters set:")
best_parameters=SVM_poly_grid.best_estimator_.get_params()
for param_name in sorted(SVM_poly_grid_parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.974
Best parameters set:
	C: 50
	degree: 1
	gamma: 0.5
	kernel: 'poly'
	random_state: 13


In [2221]:
SVM_grid = GridSearchCV(estimator = SVC(), param_grid = SVM_grid_parameters, 
                             scoring = 'accuracy')
SVM_grid.fit(cattle_train_X, cattle_train_y)
print("Best score: %0.3f" % SVM_grid.best_score_)
print("Best parameters set:")
best_parameters=SVM_grid.best_estimator_.get_params()
for param_name in sorted(SVM_grid_parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.977
Best parameters set:
	C: 50
	gamma: 0.5
	kernel: 'linear'
	random_state: 13


In [2222]:
RF_grid = GridSearchCV(estimator = RandomForestClassifier(), param_grid = RF_grid_parameters, 
                             scoring = 'accuracy')
RF_grid.fit(cattle_train_X, cattle_train_y)
print("Best score: %0.3f" % RF_grid.best_score_)
print("Best parameters set:")
best_parameters=RF_grid.best_estimator_.get_params()
for param_name in sorted(RF_grid_parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.529
Best parameters set:
	criterion: 'entropy'
	max_depth: 25
	n_estimators: 100
	random_state: 13
